In [54]:
import pandas as pd
import numpy as np 
import string

def isEnglish(s):
    try:
        s.encode('ascii')
        return 1
    except:
        return 0

data = pd.read_csv("out.csv")
data.columns
print(data.shape)

data = data[(data['educations-field_of_study'] != "--undefined--") & (data['industry'] != "--undefined--")]

columns = ["industry", "educations-degree", "educations-field_of_study", "educations-school-name", "positions-is-current", "num-connections"]
new_data = data[columns]
new_data['positions-is-current'] = new_data['positions-is-current'].replace('--undefined--', 'FALSE')
new_data = new_data.replace("--undefined--",np.NaN)
new_data = new_data.reset_index()

#print(new_data.shape[0])
is_english = []
for i in range(0, new_data.shape[0]):
    is_english.append(isEnglish(new_data["industry"][i]))
new_data["is_english"] = is_english

new_data = new_data[(new_data["is_english"] == 1)]
new_data

(888664, 16)


/Users/zenitharnejo/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,index,industry,educations-degree,educations-field_of_study,educations-school-name,positions-is-current,num-connections,is_english
0,24,Information Technology and Services,BE,Electronics and Communication,The Madras seva sadan Higher Secondary school,FALSE,1,1
1,39,Education Management,Master of Business Administration (M.B.A.),Marketing,Mumbai University,FALSE,13,1
2,45,Education Management,islamic education,islamic education,Universiti Malaya,TRUE,0,1
3,53,Public Relations and Communications,Master's,Public Communication,Institut d'Etudes politiques de Lille,FALSE,112,1
4,58,Automotive,MCA,Computer Applications,St. Joseph College Cuddalore,FALSE,18,1
5,59,Financial Services,PGDMM,Marketing,Annamalai University,TRUE,10,1
6,62,Computer Software,B.Tech,Information Technology,Anna University,TRUE,56,1
7,71,Accounting,B.COM,COMMERCE,KERALA UNIVERSITY,FALSE,2,1
8,113,"Leisure, Travel & Tourism",BSc,"physics,chemistry,maths",Mangalore University,FALSE,74,1
9,130,Information Technology and Services,Master of Technology (M.Tech.),Computer Science,Visvesvaraya Technological University,FALSE,75,1
